# Description
After running K fold to select those data which are predicted into a wrong category, we implemented mannual recorrection. This script is to apply those correction to the original files. 

In [5]:
import pandas as pd
import numpy as np

In [6]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
df_others = pd.read_csv('../../data/others/irrelevant_response_training_set.csv')
df_other_matrix = pd.read_csv('../../data/others/strategy_mat.csv')
num2text = df_other_matrix.set_index('label').category.drop_duplicates().to_dict()

df_all_original = {}
for each in model_list:
    df_all_original[each] = pd.read_csv('../../data/{}/mock_up_data1.csv'.format(each))

In [7]:
model = 'CutDebt'
df_correction = pd.read_csv('../../data/{}/labelNeedsCorrections.csv'.format(model))
other_label = max(df_all_original[model].label)
non_other_label = set(df_all_original[model].label.values) - set([other_label])
df_all_original[model] = df_all_original[model][df_all_original[model].label != other_label]
print('{} -- {}'.format(model,len(df_all_original[model])))
print('length of other: {}'.format(len(df_others)))
append_other_text = []
append_other_category = []

append_function_text = {}
append_function_category = {}

original_remove_index = {}

count_case1 = 0
count_case2 = 0
count_case3 = 0
count_case4 = 0
for index in range(len(df_correction)):
    ############ get data #####################
    cur_data = df_correction.iloc[index]
    data_from = cur_data['from']
    label = cur_data['label']
    original_text = cur_data['original_text']
    new_label = cur_data['new_label']
    ########### apply to original files #######
    if model in ['CutDebt','Installment'] and new_label == 110:
        count_case4 += 1
        if label in non_other_label:
            ### insert into others
            append_other_text.append(original_text)
            append_other_category.append(num2text[new_label])
            ### remove from original
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
        else:
            ### update others
            rindex = df_others[df_others['文本']==original_text].index.values
            if len(rindex) == 0:
                ### insert into others
                append_other_text.append(original_text)
                append_other_category.append(num2text[new_label])
            else:    
                df_others.loc[rindex,'类别'] = num2text[110]
        continue
    if label in non_other_label and new_label >= 100:
        count_case1 += 1
        ### insert into others
        append_other_text.append(original_text)
        append_other_category.append(num2text[new_label])
        
        ### remove from original
        rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
        if original_remove_index.get(data_from) is None:
            original_remove_index[data_from] = []
        original_remove_index[data_from].extend(list(rindex))
        
        
    elif label in non_other_label and new_label in non_other_label and label!= new_label:
        count_case2 += 1
        # update original data
        indexes = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
        df_all_original[data_from].loc[indexes,'label'] = new_label

    elif label == other_label and new_label in non_other_label:
        count_case3+=1
        if append_function_text.get(model) is None:
            append_function_text[model]=[]
            append_function_category[model]=[]
        append_function_text[model].append(original_text)
        append_function_category[model].append(new_label)
        
others_append = pd.DataFrame({'文本':append_other_text,'类别':append_other_category})
function_append = pd.DataFrame({'split_text':append_function_text,'label':append_function_category})

#### apply to self and others
#### drop logic
for each_key in original_remove_index:
    indexes = original_remove_index[each_key]
    df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
    print('{} -- {}'.format(each_key,len(df_all_original[each_key])))
####end drop
df_others = pd.concat([df_others,others_append],sort=True).reset_index(drop=True)
print('length of other: {}'.format(len(df_others)))
for each_key in append_function_text:
    function_append = pd.DataFrame({'split_text':append_function_text[each_key],
                                    'label':append_function_category[each_key]})
    
    df_all_original[each_key] = pd.concat([df_all_original[each_key],function_append],sort=True).reset_index(drop=True)
    print('{} -- {}'.format(each_key,len(df_all_original[each_key])))

print('case 1 has data : {}'.format(count_case1))
print('case 2 has data : {}'.format(count_case2))
print('case 3 has data : {}'.format(count_case3))
print('case 4 has data : {}'.format(count_case4))


CutDebt -- 4128
length of other: 4695
CutDebt -- 3874
length of other: 4936
CutDebt -- 4168
case 1 has data : 225
case 2 has data : 94
case 3 has data : 294
case 4 has data : 21


In [8]:
others_append

,文本,类别
0,我正在想办法,模糊确认
1,一会去办理,模糊确认
2,明天,模糊确认
3,我自己处理,模糊确认
4,你等着看好了,模糊确认
5,尽量,模糊确认
6,只愿还本金,讨价还价
7,知道了,模糊确认
8,信号不好，没听清,请求重复
9,多少多少我就还,讨价还价


In [71]:
original_remove_index

{'ConfirmLoan': [1304,
  1287,
  973,
  1115,
  1179,
  372,
  402,
  1113,
  1074,
  1381,
  332,
  285,
  97,
  1314,
  1406,
  127,
  983,
  315,
  178,
  1045,
  180,
  1341,
  1356,
  1294,
  1020]}

In [58]:
df_correction

,from,label,original_label,original_text,new_label
0,讨价还价,2,100.0,违约金太多了,2.0
1,ConfirmLoan,0,NaN,你不清楚你们是高利贷么？还敢给我打电话,0.0
2,确认数额,2,102.0,怎么这么多,2.0
3,ConfirmLoan,0,NaN,我也是想还你们，可惜是心有余而力不足，现实太残酷，一分钱难倒英雄汉,0.0
4,故意岔开话题,2,109.0,起床了,2.0
5,故意岔开话题,2,109.0,阿根廷进球了,2.0
6,还款方式,2,108.0,怎么还款我给忘了,2.0
7,ConfirmLoan,1,NaN,啥子哦？你有没有搞错,1.0
8,ConfirmLoan,0,NaN,已经好几天没吃饭了,0.0
9,ConfirmLoan,0,NaN,我到处都欠款，你说我该还你们谁啊,0.0


In [42]:
len(others_append)

25

In [35]:
others_append.drop(index=[3,4,5])

,文本,类别
0,别有事没事打电话,故意岔开话题
1,电话费不要钱啊？天天乱打。,故意岔开话题
2,我就等你们打电话来，告诉我该怎么还，要不都不知道去哪里还,还款方式
6,操,故意岔开话题
7,你继续扯经。,故意岔开话题
8,我欠了多少钱？,确认数额
9,我欠了你们多少钱？你说说看,确认数额
10,我现在还欠哪个的钱哦,确认数额
11,你放高利贷啊，这么多,确认数额
12,最怕江南游子再故地重游,故意岔开话题


In [12]:
df_correction.iloc[0]

from                讨价还价
label                  2
original_label       100
original_text     违约金太多了
new_label              2
Name: 0, dtype: object

In [4]:
df_correction

,from,label,original_label,original_text,new_label
0,讨价还价,2,100.0,违约金太多了,2.0
1,ConfirmLoan,0,NaN,你不清楚你们是高利贷么？还敢给我打电话,0.0
2,确认数额,2,102.0,怎么这么多,2.0
3,ConfirmLoan,0,NaN,我也是想还你们，可惜是心有余而力不足，现实太残酷，一分钱难倒英雄汉,0.0
4,故意岔开话题,2,109.0,起床了,2.0
5,故意岔开话题,2,109.0,阿根廷进球了,2.0
6,还款方式,2,108.0,怎么还款我给忘了,2.0
7,ConfirmLoan,1,NaN,啥子哦？你有没有搞错,1.0
8,ConfirmLoan,0,NaN,已经好几天没吃饭了,0.0
9,ConfirmLoan,0,NaN,我到处都欠款，你说我该还你们谁啊,0.0
